In [2]:
import numpy as np # Linear Algebra
import matplotlib.pyplot as plt # Plotting

In [3]:
def RotX(theta):
    ct = np.cos(theta)
    st = np.sin(theta)
    R = np.eye(3, 3)
    R[1, 1] = ct
    R[1, 2] = -st
    R[2, 1] = st
    R[2, 2] = ct
    return R

def RotY(theta):
    ct = np.cos(theta)
    st = np.sin(theta)
    R = np.eye(3, 3)
    R[0, 0] = ct
    R[0, 2] = st
    R[2, 0] = -st
    R[2, 2] = ct
    return R

def RotZ(theta):
    ct = np.cos(theta)
    st = np.sin(theta)
    R = np.eye(3, 3)
    R[0, 0] = ct
    R[0, 1] = -st
    R[1, 0] = st
    R[1, 1] = ct
    return R

In [4]:
def hat(vec):
    v = vec.reshape((3,))
    return np.array([
        [0., -v[2], v[1]],
        [v[2], 0., -v[0]],
        [-v[1], v[0], 0.]
    ])

def angular_velocity_to_deriv_rotation(omega_w, R):
    return hat(omega_w) @ R

def angular_velocity_local_to_deriv_rotation(omega_b, R):
    return R @ hat(omega_b)

In [5]:
# Some testing
R = RotX(np.pi / 2.)

omega_b = np.array([[1., 0., 0.]]).T

# Integrate naively for a few steps
dt = 0.1
R1 = np.copy(R)
R2 = np.copy(R)

for i in range(5):
    omega_w = R1 @ omega_b # rotate local omega to world frame
    R1d = angular_velocity_to_deriv_rotation(omega_w, R1)
    R2d = angular_velocity_local_to_deriv_rotation(omega_b, R2)

    R1 = R1 + R1d * dt # Euler integration!
    R2 = R2 + R2d * dt # Euler integration!

# These two should be very similar
print(R1)
print("========================")
print(R2)
print("========================")
print(np.linalg.norm(R1-R2))

[[ 1.       0.       0.     ]
 [ 0.      -0.49001 -0.9005 ]
 [ 0.       0.9005  -0.49001]]
[[ 1.       0.       0.     ]
 [ 0.      -0.49001 -0.9005 ]
 [ 0.       0.9005  -0.49001]]
0.0


In [6]:
def homogeneous(R, p = np.zeros((3, 1))):
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3:] = p

    return T

def skew_velocity(V):
    omega = V[:3, :]
    v  = V[3:, :]

    T = np.zeros(shape=(4, 4))
    T[:3, :3] = hat(omega)
    T[:3, 3:] = v

    return T

def spatial_velocity_to_deriv_homogeneous(Vw, T):
    return skew_velocity(Vw) @ T

def spatial_velocity_local_to_deriv_homogeneous(Vb, T):
    return T @ skew_velocity(Vb)

In [7]:
# Let's see if this is the same as pure Rotation kinematics
R = RotX(np.pi / 2.)

omega_b = np.array([[1., 0., 0.]]).T

T = homogeneous(R, np.zeros((3, 1)))

Vb = np.vstack([omega_b, np.zeros((3, 1))])

# Integrate naively for a few steps
dt = 0.1
T1 = np.copy(T)
T2 = np.copy(T)

for i in range(5):
    omega_w = T1[:3, :3] @ omega_b # rotate local omega to world frame
    Vw = np.vstack([omega_w, np.zeros((3, 1))])
    T1d = spatial_velocity_to_deriv_homogeneous(Vw, T1)
    T2d = spatial_velocity_local_to_deriv_homogeneous(Vb, T2)

    T1 = T1 + T1d * dt # Euler integration!
    T2 = T2 + T2d * dt # Euler integration!

# These two should be very similar
print(T1)
print("========================")
print(T2)
print("========================")
print(np.linalg.norm(T1-T2))

[[ 1.       0.       0.       0.     ]
 [ 0.      -0.49001 -0.9005   0.     ]
 [ 0.       0.9005  -0.49001  0.     ]
 [ 0.       0.       0.       1.     ]]
[[ 1.       0.       0.       0.     ]
 [ 0.      -0.49001 -0.9005   0.     ]
 [ 0.       0.9005  -0.49001  0.     ]
 [ 0.       0.       0.       1.     ]]
0.0


In [8]:
# Adjoint Representation
def adjoint(R, p):
    Tadj = np.zeros((6, 6))
    Tadj[:3, :3] = R
    Tadj[3:, 3:] = R
    Tadj[3:, :3] = hat(p) @ R

    return Tadj

In [9]:
# Let's do a few tests
Vb = np.array([[1., 2., 0.1, 2., -1., 0.]]).T

R = RotY(np.pi / 2.)
t = np.array([[0., 1., 0.]]).T

Vw = adjoint(R, t) @ Vb

Vbn = adjoint(R.T, -R.T@t) @ Vw

print(Vb)
print(Vw)
print(Vbn)

[[ 1. ]
 [ 2. ]
 [ 0.1]
 [ 2. ]
 [-1. ]
 [ 0. ]]
[[ 0.1]
 [ 2. ]
 [-1. ]
 [-1. ]
 [-1. ]
 [-2.1]]
[[ 1.00000000e+00]
 [ 2.00000000e+00]
 [ 1.00000000e-01]
 [ 2.00000000e+00]
 [-1.00000000e+00]
 [-1.75656114e-17]]
